In [227]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [228]:
# 패키지 설치
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, precision_score, recall_score, auc
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.metrics import classification_report, accuracy_score, f1_score

from bayes_opt import BayesianOptimization

In [229]:
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import warnings
warnings.filterwarnings('ignore')

In [231]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import TomekLinks
from collections import Counter

In [232]:
final = pd.read_csv('/Users/sungahwang/Desktop/BC_data/final.csv')
final.head()

,user_id,sum_applied,application_id,gender,insert_time,yearly_income,income_type,employment_type,houseown_type,desired_amount,...,is_applied,weekday,loanapply_insert_hour,bank_apply_rate,product_apply_rate,cluster2,action_cnt,use_day_cnt,first_event,timeout_sum(s)
0,545882.0,8.0,576643,1.0,2022-05-09 10:54:51,72000000.0,직장가입자(4대보험O),정규직,전월세,20000000.0,...,0.0,0,10,0.042501,0.050284,0,41.913806,5.765978,CompleteIDCertification,67.997282
1,545882.0,8.0,576643,1.0,2022-05-09 10:54:51,72000000.0,직장가입자(4대보험O),정규직,전월세,20000000.0,...,0.0,0,10,0.017018,0.016149,0,41.913806,5.765978,CompleteIDCertification,67.997282
2,558819.0,5.0,2136706,1.0,2022-05-09 10:41:05,39000000.0,직장가입자(4대보험O),정규직,전월세,80000000.0,...,0.0,0,10,0.024636,0.028622,0,41.913806,5.765978,CompleteIDCertification,67.997282
3,558819.0,5.0,2136706,1.0,2022-05-09 10:41:05,39000000.0,직장가입자(4대보험O),정규직,전월세,80000000.0,...,0.0,0,10,0.021634,0.035367,0,41.913806,5.765978,CompleteIDCertification,67.997282
4,558819.0,5.0,2136706,1.0,2022-05-09 10:41:05,39000000.0,직장가입자(4대보험O),정규직,전월세,80000000.0,...,0.0,0,10,0.027749,0.035265,4,41.913806,5.765978,CompleteIDCertification,67.997282


In [233]:
final = final.drop(['user_id', 'application_id', 'insert_time', 'loanapply_insert_time'], axis = 1)

In [234]:
final.isnull().sum()

sum_applied                                  0
gender                                       0
yearly_income                                0
income_type                                  0
employment_type                              0
houseown_type                                0
desired_amount                               0
purpose                                      0
personal_rehabilitation_yn                   0
personal_rehabilitation_complete_yn          0
existing_loan_cnt                            0
existing_loan_amt                            0
reage                                        0
career                                       0
credit_grade                                 0
spec_clust                                   0
bank_id                                      0
product_id                                   0
loan_limit                                   0
loan_rate                                    0
is_applied                             3255482
weekday      

In [235]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13141813 entries, 0 to 13141812
Data columns (total 30 columns):
 #   Column                               Dtype  
---  ------                               -----  
 0   sum_applied                          float64
 1   gender                               float64
 2   yearly_income                        float64
 3   income_type                          object 
 4   employment_type                      object 
 5   houseown_type                        object 
 6   desired_amount                       float64
 7   purpose                              object 
 8   personal_rehabilitation_yn           float64
 9   personal_rehabilitation_complete_yn  float64
 10  existing_loan_cnt                    float64
 11  existing_loan_amt                    float64
 12  reage                                float64
 13  career                               float64
 14  credit_grade                         float64
 15  spec_clust                    

In [236]:
final['income_type'] = final['income_type'].astype('category')
final['employment_type'] = final['employment_type'].astype('category')
final['houseown_type'] = final['houseown_type'].astype('category')
final['purpose'] = final['purpose'].astype('category')
final['first_event'] = final['first_event'].astype('category')
final['weekday'] = final['weekday'].astype('category')
final['personal_rehabilitation_yn'] = final['personal_rehabilitation_yn'].astype('category')
final['personal_rehabilitation_complete_yn'] = final['personal_rehabilitation_complete_yn'].astype('category')
final['gender'] = final['gender'].astype('category')
final['spec_clust'] = final['spec_clust'].astype('category')
final['cluster2'] = final['cluster2'].astype('category')

In [237]:
final['yearly_income']=np.log(final['yearly_income'])
final["desired_amount"]=np.log(final["desired_amount"])
final["existing_loan_amt"]=np.log(final["existing_loan_amt"])

In [238]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13141813 entries, 0 to 13141812
Data columns (total 30 columns):
 #   Column                               Dtype   
---  ------                               -----   
 0   sum_applied                          float64 
 1   gender                               category
 2   yearly_income                        float64 
 3   income_type                          category
 4   employment_type                      category
 5   houseown_type                        category
 6   desired_amount                       float64 
 7   purpose                              category
 8   personal_rehabilitation_yn           category
 9   personal_rehabilitation_complete_yn  category
 10  existing_loan_cnt                    float64 
 11  existing_loan_amt                    float64 
 12  reage                                float64 
 13  career                               float64 
 14  credit_grade                         float64 
 15  spec_clust   

In [239]:
final = pd.get_dummies(final)
final

,sum_applied,yearly_income,desired_amount,existing_loan_cnt,existing_loan_amt,reage,career,credit_grade,bank_id,product_id,...,cluster2_1,cluster2_2,cluster2_3,cluster2_4,cluster2_5,first_event_CompleteIDCertification,first_event_EndLoanApply,first_event_GetCreditInfo,first_event_StartLoanApply,first_event_others
0,8.0,18.092177,16.811243,7.0,18.478950,40.0,4.0,7.0,54,235,...,0,0,0,0,0,1,0,0,0,0
1,8.0,18.092177,16.811243,7.0,18.478950,40.0,4.0,7.0,11,118,...,0,0,0,0,0,1,0,0,0,0
2,5.0,17.479072,18.197537,2.0,17.875954,40.0,11.0,5.0,42,216,...,0,0,0,0,0,1,0,0,0,0
3,5.0,17.479072,18.197537,2.0,17.875954,40.0,11.0,5.0,25,169,...,0,0,0,0,0,1,0,0,0,0
4,5.0,17.479072,18.197537,2.0,17.875954,40.0,11.0,5.0,22,221,...,0,0,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13141808,4.0,18.683045,18.420681,0.0,-inf,40.0,11.0,3.0,60,183,...,0,0,0,0,0,1,0,0,0,0
13141809,14.0,17.974394,16.811243,8.0,18.832790,40.0,3.0,6.0,2,7,...,0,0,0,0,0,1,0,0,0,0
13141810,6.0,17.875954,14.914123,1.0,17.588271,40.0,16.0,7.0,59,150,...,0,0,0,0,0,1,0,0,0,0
13141811,2.0,17.370859,15.607270,3.0,17.111347,40.0,2.0,7.0,49,136,...,0,0,0,0,0,1,0,0,0,0


In [240]:
final_train = final[final['is_applied'].notnull()]
final_test = final[final['is_applied'].isnull()]

In [242]:
data = final_train.drop('is_applied',axis=1)
target = final_train['is_applied']

In [243]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(data, target, test_size=0.3, random_state=0)

# lightGBM 기본

In [244]:
lgb = LGBMClassifier()
lgb.fit(X_train, y_train)
lgb_preds= lgb.predict(X_val)

In [245]:
lgb_preds

array([0., 0., 0., ..., 0., 0., 0.])

In [246]:
#f1 score
print('f1 score : {:0.4f}'.format(f1_score(y_val, lgb_preds)))
#정확도
print('accuracy: {:0.4f}'.format(accuracy_score(y_val, lgb_preds)))
# 정밀도
print('precision: {:0.4f}'.format(precision_score(y_val, lgb_preds)))
#재현율
print('recall: {:0.4f}'.format(recall_score(y_val,lgb_preds)))
# ROC_AUC
print('roc_auc: {:0.4f}'.format(roc_auc_score(y_val, lgb_preds)))

f1 score : 0.1300
accuracy: 0.9448
precision: 0.6118
recall: 0.0727
roc_auc: 0.5350


# Random Under Sampling + lgb 기본

In [247]:
# 모델 설정
rus = RandomUnderSampler(random_state= 42,sampling_strategy= {0:600000, 1:392199 })

In [248]:
X_rus, y_rus = rus.fit_resample(X_train, y_train)
print(Counter(y_rus))

Counter({0.0: 600000, 1.0: 392199})


In [249]:
lgb_rus = LGBMClassifier()
lgb_rus.fit(X_rus, y_rus)
lgb_rus_preds= lgb_rus.predict(X_val)

In [250]:
#f1 score
print('f1 score : {:0.4f}'.format(f1_score(y_val, lgb_rus_preds)))
#정확도
print('accuracy: {:0.4f}'.format(accuracy_score(y_val, lgb_rus_preds)))
# 정밀도
print('precision: {:0.4f}'.format(precision_score(y_val, lgb_rus_preds)))
#재현율
print('recall: {:0.4f}'.format(recall_score(y_val,lgb_rus_preds)))
# ROC_AUC
print('roc_auc: {:0.4f}'.format(roc_auc_score(y_val, lgb_rus_preds)))

f1 score : 0.3380
accuracy: 0.8254
precision: 0.2153
recall: 0.7861
roc_auc: 0.8070


#### 파라미터 최적화

In [261]:
lgb_param = LGBMClassifier(scale_pos_weight = 8)
lgb_param.fit(X_train, y_train)
lgb_param_preds= lgb_param.predict(X_val)

In [262]:
#f1 score
print('f1 score : {:0.4f}'.format(f1_score(y_val, lgb_param_preds)))
#정확도
print('accuracy: {:0.4f}'.format(accuracy_score(y_val, lgb_param_preds)))
# 정밀도
print('precision: {:0.4f}'.format(precision_score(y_val, lgb_param_preds)))
#재현율
print('recall: {:0.4f}'.format(recall_score(y_val,lgb_param_preds)))
# ROC_AUC
print('roc_auc: {:0.4f}'.format(roc_auc_score(y_val, lgb_param_preds)))

f1 score : 0.3647
accuracy: 0.8583
precision: 0.2445
recall: 0.7176
roc_auc: 0.7922


In [263]:
lgb_param = LGBMClassifier(scale_pos_weight = 8)
lgb_param.fit(X_rus, y_rus)
lgb_param_preds= lgb_param.predict(X_val)

In [264]:
#f1 score
print('f1 score : {:0.4f}'.format(f1_score(y_val, lgb_param_preds)))
#정확도
print('accuracy: {:0.4f}'.format(accuracy_score(y_val, lgb_param_preds)))
# 정밀도
print('precision: {:0.4f}'.format(precision_score(y_val, lgb_param_preds)))
#재현율
print('recall: {:0.4f}'.format(recall_score(y_val,lgb_param_preds)))
# ROC_AUC
print('roc_auc: {:0.4f}'.format(roc_auc_score(y_val, lgb_param_preds)))

f1 score : 0.1992
accuracy: 0.5510
precision: 0.1108
recall: 0.9852
roc_auc: 0.7551


# random search

In [268]:
parameters = {'num_leaves':[20,40,60,80], 'min_child_samples':[5,10,15],'max_depth':[5,10,20],
             'learning_rate':[0.05,0.1,0.2], 'scale_pos_weight':[5,8,11]}

In [271]:
from sklearn.model_selection import RandomizedSearchCV
clf=RandomizedSearchCV(lgb,parameters,scoring='precision',n_iter=10)
clf.fit(X=X_train, y=y_train)
print(clf.best_params_)

{'scale_pos_weight': 5, 'num_leaves': 80, 'min_child_samples': 5, 'max_depth': 20, 'learning_rate': 0.1}


In [274]:
lgb_param = LGBMClassifier(scale_pos_weight = 5, num_leaves = 80, min_child_samples = 5, max_depth = 20, learning_rate = 0.1)
lgb_param.fit(X_train, y_train)
lgb_param_preds= lgb_param.predict(X_train)

In [275]:
#f1 score
print('f1 score : {:0.4f}'.format(f1_score(y_train, lgb_param_preds)))
#정확도
print('accuracy: {:0.4f}'.format(accuracy_score(y_train, lgb_param_preds)))
# 정밀도
print('precision: {:0.4f}'.format(precision_score(y_train, lgb_param_preds)))
#재현율
print('recall: {:0.4f}'.format(recall_score(y_train,lgb_param_preds)))
# ROC_AUC
print('roc_auc: {:0.4f}'.format(roc_auc_score(y_train, lgb_param_preds)))

f1 score : 0.4080
accuracy: 0.9022
precision: 0.3105
recall: 0.5946
roc_auc: 0.7577


In [272]:
lgb_param = LGBMClassifier(scale_pos_weight = 5, num_leaves = 80, min_child_samples = 5, max_depth = 20, learning_rate = 0.1)
lgb_param.fit(X_train, y_train)
lgb_param_preds= lgb_param.predict(X_val)

In [273]:
#f1 score
print('f1 score : {:0.4f}'.format(f1_score(y_val, lgb_param_preds)))
#정확도
print('accuracy: {:0.4f}'.format(accuracy_score(y_val, lgb_param_preds)))
# 정밀도
print('precision: {:0.4f}'.format(precision_score(y_val, lgb_param_preds)))
#재현율
print('recall: {:0.4f}'.format(recall_score(y_val,lgb_param_preds)))
# ROC_AUC
print('roc_auc: {:0.4f}'.format(roc_auc_score(y_val, lgb_param_preds)))

f1 score : 0.4047
accuracy: 0.9018
precision: 0.3082
recall: 0.5890
roc_auc: 0.7548


* 현재까지 best

In [276]:
parameters2 = {'num_leaves':[70,80,90,100], 'min_child_samples':[3,4,5,6],'max_depth':[15,20,25],
             'learning_rate':[0.05,0.1,0.15], 'scale_pos_weight':[3,5,6,8]}

In [277]:
clf=RandomizedSearchCV(lgb,parameters2,scoring='precision',n_iter=10)
clf.fit(X=X_train, y=y_train)
print(clf.best_params_)

{'scale_pos_weight': 3, 'num_leaves': 100, 'min_child_samples': 3, 'max_depth': 25, 'learning_rate': 0.05}


In [283]:
lgb_param2 = LGBMClassifier(scale_pos_weight = 3, num_leaves = 100, min_child_samples = 3, max_depth = 25, learning_rate = 0.05)
lgb_param2.fit(X_train, y_train)
lgb_param_preds2= lgb_param2.predict(X_val)

In [284]:
#f1 score
print('f1 score : {:0.4f}'.format(f1_score(y_val, lgb_param_preds2)))
#정확도
print('accuracy: {:0.4f}'.format(accuracy_score(y_val, lgb_param_preds2)))
# 정밀도
print('precision: {:0.4f}'.format(precision_score(y_val, lgb_param_preds2)))
#재현율
print('recall: {:0.4f}'.format(recall_score(y_val,lgb_param_preds2)))
# ROC_AUC
print('roc_auc: {:0.4f}'.format(roc_auc_score(y_val, lgb_param_preds2)))

f1 score : 0.3856
accuracy: 0.9313
precision: 0.3913
recall: 0.3800
roc_auc: 0.6722


* f1으로

In [ ]:
parameters = {'num_leaves':[20,40,60,80], 'min_child_samples':[5,10,15],'max_depth':[5,10,20],
             'learning_rate':[0.05,0.1,0.2], 'scale_pos_weight':[5,8,11]}

In [285]:
clf=RandomizedSearchCV(lgb,parameters,scoring='f1',n_iter=10)
clf.fit(X=X_train, y=y_train)
print(clf.best_params_)

{'scale_pos_weight': 5, 'num_leaves': 60, 'min_child_samples': 15, 'max_depth': 20, 'learning_rate': 0.1}


In [286]:
lgb_param3 = LGBMClassifier(scale_pos_weight = 5, num_leaves = 60, min_child_samples = 15, max_depth = 20, learning_rate = 0.1)
lgb_param3.fit(X_train, y_train)
lgb_param_preds3= lgb_param3.predict(X_val)

In [287]:
#f1 score
print('f1 score : {:0.4f}'.format(f1_score(y_val, lgb_param_preds3)))
#정확도
print('accuracy: {:0.4f}'.format(accuracy_score(y_val, lgb_param_preds3)))
# 정밀도
print('precision: {:0.4f}'.format(precision_score(y_val, lgb_param_preds3)))
#재현율
print('recall: {:0.4f}'.format(recall_score(y_val,lgb_param_preds3)))
# ROC_AUC
print('roc_auc: {:0.4f}'.format(roc_auc_score(y_val, lgb_param_preds3)))

f1 score : 0.4019
accuracy: 0.9012
precision: 0.3059
recall: 0.5858
roc_auc: 0.7530


In [288]:
parameters3 = {'num_leaves':[50, 60, 70], 'min_child_samples':[13,15,17],'max_depth':[15,20,25],
             'learning_rate':[0.05,0.1,0.15], 'scale_pos_weight':[4,5,6,8]}

In [289]:
clf=RandomizedSearchCV(lgb,parameters3,scoring='f1',n_iter=10)
clf.fit(X=X_train, y=y_train)
print(clf.best_params_)

{'scale_pos_weight': 4, 'num_leaves': 70, 'min_child_samples': 13, 'max_depth': 25, 'learning_rate': 0.15}


In [290]:
lgb_param4 = LGBMClassifier(scale_pos_weight = 4, num_leaves = 70, min_child_samples = 13, max_depth = 25, learning_rate = 0.15)
lgb_param4.fit(X_train, y_train)
lgb_param_preds4= lgb_param4.predict(X_val)

In [291]:
#f1 score
print('f1 score : {:0.4f}'.format(f1_score(y_val, lgb_param_preds4)))
#정확도
print('accuracy: {:0.4f}'.format(accuracy_score(y_val, lgb_param_preds4)))
# 정밀도
print('precision: {:0.4f}'.format(precision_score(y_val, lgb_param_preds4)))
#재현율
print('recall: {:0.4f}'.format(recall_score(y_val,lgb_param_preds4)))
# ROC_AUC
print('roc_auc: {:0.4f}'.format(roc_auc_score(y_val, lgb_param_preds4)))

f1 score : 0.4105
accuracy: 0.9158
precision: 0.3403
recall: 0.5174
roc_auc: 0.7285


## rus + random search

In [295]:
parameters4 = {'num_leaves':[30, 60, 70, 100], 'min_child_samples':[10,15,20],'max_depth':[10,20,30],
             'learning_rate':[0.05,0.1,0.15], 'scale_pos_weight':[5,6,8,11]}

In [ ]:
clf=RandomizedSearchCV(lgb,parameters4,scoring='f1',n_iter=10)
clf.fit(X=X_rus, y=y_rus)
print(clf.best_params_)

In [293]:
lgb_param5 = LGBMClassifier(scale_pos_weight = 5, num_leaves = 80, min_child_samples = 15, max_depth = 5, learning_rate = 0.2)
lgb_param5.fit(X_rus, y_rus)
lgb_param_preds5= lgb_param5.predict(X_val)

In [ ]:
#f1 score
print('f1 score : {:0.4f}'.format(f1_score(y_val, lgb_param_preds5)))
#정확도
print('accuracy: {:0.4f}'.format(accuracy_score(y_val, lgb_param_preds5)))
# 정밀도
print('precision: {:0.4f}'.format(precision_score(y_val, lgb_param_preds5)))
#재현율
print('recall: {:0.4f}'.format(recall_score(y_val,lgb_param_preds5)))
# ROC_AUC
print('roc_auc: {:0.4f}'.format(roc_auc_score(y_val, lgb_param_preds5)))

# 베이지안 시도

In [197]:
import lightgbm

def lgb_classifier (num_leaves, max_depth, 
                   lambda_l2, lambda_l1, min_child_samples, 
                   min_data_in_leaf, scale_pos_weight):
    params = {"boosting_type": 'gbdt',
              "objective" : "binary",
              "metric" : 'f1',  
              "num_leaves" : int(num_leaves),
              "max_depth" : int(max_depth),
              "lambda_l2" : lambda_l2,
              "lambda_l1" : lambda_l1,
              "num_threads" : 20,
              "min_child_samples" : int(min_child_samples),
              "min_data_in_leaf": int(min_data_in_leaf),
              "learning_rate" : 0.01,
              "subsample_freq" : 5,
              "bagging_seed" : 42,
              "verbosity" : -1,
              "scale_pos_weight" : int(scale_pos_weight),
              }
    
    lgb_param.fit(X_train, y_train)
    lgb_param_preds= lgb_param.predict(X_val)
    f1 = f1_score(y_val, lgb_param_preds)

    return f1

In [198]:
lgbBO = BayesianOptimization(lgb_classifier, { 'num_leaves': (25, 4000),
                                                'max_depth': (5, 40),
                                                'lambda_l2': (0.0, 0.1),
                                                'lambda_l1': (0.0, 0.1),
                                                'min_child_samples': (50, 10000),
                                                'min_data_in_leaf': (100, 2000),
                                                'scale_pos_weight': (3,7),
                                                })

lgbBO.maximize(n_iter=15, init_points=2)

|   iter    |  target   | lambda_l1 | lambda_l2 | max_depth | min_ch... | min_da... | num_le... | scale_... |
-------------------------------------------------------------------------------------------------------------
[LightGBM] [Warning] num_threads is set=20, n_jobs=-1 will be ignored. Current value: num_threads=20
[LightGBM] [Warning] lambda_l1 is set=0.03378, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.03378
[LightGBM] [Warning] min_data_in_leaf is set=254, min_child_samples=348 will be ignored. Current value: min_data_in_leaf=254
[LightGBM] [Warning] lambda_l2 is set=0.09738, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.09738
|  1        |  0.1855   |  0.04414  |  0.0596   |  11.29    |  2.135e+0 |  215.6    |  676.9    |  6.222    |
[LightGBM] [Warning] num_threads is set=20, n_jobs=-1 will be ignored. Current value: num_threads=20
[LightGBM] [Warning] lambda_l1 is set=0.03378, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.03378
[LightGBM

TypeError: 'float' object is not subscriptable

In [192]:
def modelFitter(colsampleByTree, subsample,maxDepth, num_leaves,learning_rate,min_child_weight):
    
    model = LGBMClassifier(learning_rate=learning_rate, n_estimators=10000, max_depth=maxDepth.astype("int32"), subsample=subsample, colsample_bytree=colsampleByTree,num_leaves=num_leaves.astype("int32"),min_child_weight=min_child_weight)

    model.fit(X_train, y_train, eval_metric="F1", early_stopping_rounds=50, verbose=False)
    model_preds= model.predict(X_val)
    f1 = f1_score(X_val, model_preds)

    return f1

# Bounded region of parameter space
pbounds = {'colsampleByTree': (0.5,1.0), 'subsample': (0.8,1.0), 'maxDepth': (3,10), 'num_leaves': (24, 45),'learning_rate':(0.01,0.3),'min_child_weight':(5,9)}

optimizer = BayesianOptimization(
    f=modelFitter,
    pbounds=pbounds,
    random_state=1)

optimizer.maximize(init_points=5,n_iter=5)

|   iter    |  target   | colsam... | learni... | maxDepth  | min_ch... | num_le... | subsample |
-------------------------------------------------------------------------------------------------


ValueError: For early stopping, at least one dataset and eval metric is required for evaluation

In [150]:
from sklearn.metrics import f1_score

def evaluate_macroF1_lgb(truth, predictions):  

    # this follows the discussion in https://github.com/Microsoft/LightGBM/issues/1483

    pred_labels = predictions.reshape(len(np.unique(truth)),-1).argmax(axis=0)

    f1 = f1_score(truth, pred_labels, average='macro')

    return ('macroF1', f1, True) 

In [153]:
import lightgbm

def lgb_classifier (num_leaves, max_depth, 
                   lambda_l2, lambda_l1, min_child_samples, 
                   min_data_in_leaf, scale_pos_weight):
    params = {"boosting_type": 'gbdt',
              "objective" : "binary",
              "metric" : evaluate_macroF1_lgb,  
              "num_leaves" : int(num_leaves),
              "max_depth" : int(max_depth),
              "lambda_l2" : lambda_l2,
              "lambda_l1" : lambda_l1,
              "num_threads" : 20,
              "min_child_samples" : int(min_child_samples),
              "min_data_in_leaf": int(min_data_in_leaf),
              "learning_rate" : 0.01,
              "subsample_freq" : 5,
              "bagging_seed" : 42,
              "verbosity" : -1,
              "scale_pos_weight" : int(scale_pos_weight),
              }
    train_data = lightgbm.Dataset(X_train, y_train)
    cv_result = lightgbm.cv(params,
                       train_data,
                       1000,
                       early_stopping_rounds=100,
                       stratified=True,
                       nfold=3)
    return cv_result['f1-mean'][-1]

In [154]:
lgbBO = BayesianOptimization(lgb_classifier, { 'num_leaves': (25, 4000),
                                                'max_depth': (5, 40),
                                                'lambda_l2': (0.0, 0.1),
                                                'lambda_l1': (0.0, 0.1),
                                                'min_child_samples': (50, 10000),
                                                'min_data_in_leaf': (100, 2000),
                                                'scale_pos_weight': (3,7),
                                                })

lgbBO.maximize(n_iter=15, init_points=2)

|   iter    |  target   | lambda_l1 | lambda_l2 | max_depth | min_ch... | min_da... | num_le... | scale_... |
-------------------------------------------------------------------------------------------------------------


TypeError: Unknown type of parameter:metric, got:function

In [24]:
# 모델 설정
rus = RandomUnderSampler(random_state=0, sampling_strategy='majority')

In [25]:
X_rus, y_rus = rus.fit_resample(X_train, y_train)
print(Counter(y_rus))

Counter({0.0: 392199, 1.0: 392199})


In [26]:
lgb_rus = LGBMClassifier()
lgb_rus.fit(X_rus, y_rus)
lgb_rus_preds= lgb_rus.predict(X_val)

In [27]:
#f1 score
print('f1 score : {:0.4f}'.format(f1_score(y_val, lgb_rus_preds)))
#정확도
print('accuracy: {:0.4f}'.format(accuracy_score(y_val, lgb_rus_preds)))
# 정밀도
print('precision: {:0.4f}'.format(precision_score(y_val, lgb_rus_preds)))
#재현율
print('recall: {:0.4f}'.format(recall_score(y_val,lgb_rus_preds)))
# ROC_AUC
print('roc_auc: {:0.4f}'.format(roc_auc_score(y_val, lgb_rus_preds)))

f1 score : 0.2999
accuracy: 0.7703
precision: 0.1812
recall: 0.8678
roc_auc: 0.8161


# Under Sampling - TomekLinks + lgb 기본

tl = TomekLinks()

X_tl, y_tl = tl.fit_resample(X_train, y_train)
print(Counter(y_tl))

lgb_tl = LGBMClassifier()
lgb_tl.fit(X_tl, y_tl)
lgb_tl_preds= lgb_tl.predict(X_val)

#f1 score
print('f1 score : {:0.4f}'.format(f1_score(y_val, lgb_tl_preds)))
#정확도
print('accuracy: {:0.4f}'.format(accuracy_score(y_val, lgb_tl_preds)))
#정밀도
print('precision: {:0.4f}'.format(precision_score(y_val, lgb_tl_preds)))
#재현율
print('recall: {:0.4f}'.format(recall_score(y_val,lgb_tl_preds)))
#ROC_AUC
print('roc_auc: {:0.4f}'.format(roc_auc_score(y_val, lgb_tl_preds)))

# Random Over Sampling + lgb 기본

In [31]:
# 모델 설정
ros = RandomOverSampler(random_state=42)

In [ ]:
X_ros, y_ros = ros.fit_resample(X_train, y_train)
print(Counter(y_ros))

In [33]:
lgb_ros = LGBMClassifier()
lgb_ros.fit(X_ros, y_ros)
lgb_ros_preds= lgb_ros.predict(X_val)

In [34]:
#f1 score
print('f1 score : {:0.4f}'.format(f1_score(y_val, lgb_ros_preds)))
#정확도
print('accuracy: {:0.4f}'.format(accuracy_score(y_val, lgb_ros_preds)))
# 정밀도
print('precision: {:0.4f}'.format(precision_score(y_val, lgb_ros_preds)))
#재현율
print('recall: {:0.4f}'.format(recall_score(y_val,lgb_ros_preds)))
# ROC_AUC
print('roc_auc: {:0.4f}'.format(roc_auc_score(y_val, lgb_ros_preds)))

f1 score : 0.3005
accuracy: 0.7715
precision: 0.1818
recall: 0.8660
roc_auc: 0.8159


# catboost + rus

In [85]:
# 모델 설정
rus = RandomUnderSampler(random_state= 42, sampling_strategy= {0:600000, 1:392199 })

In [86]:
X_rus, y_rus = rus.fit_resample(X_train, y_train)
print(Counter(y_rus))

Counter({0.0: 600000, 1.0: 392199})


In [87]:
cb_rus = CatBoostClassifier()
cb_rus.fit(X_rus, y_rus)
cb_rus_preds= cb_rus.predict(X_val)

Learning rate set to 0.196102
0:	learn: 0.5680940	total: 68.7ms	remaining: 1m 8s
1:	learn: 0.5220456	total: 122ms	remaining: 1m 1s
2:	learn: 0.5003818	total: 178ms	remaining: 59.1s
3:	learn: 0.4857481	total: 233ms	remaining: 58.1s
4:	learn: 0.4758343	total: 288ms	remaining: 57.2s
5:	learn: 0.4675355	total: 343ms	remaining: 56.7s
6:	learn: 0.4595177	total: 399ms	remaining: 56.6s
7:	learn: 0.4535529	total: 452ms	remaining: 56s
8:	learn: 0.4492149	total: 511ms	remaining: 56.3s
9:	learn: 0.4447054	total: 576ms	remaining: 57s
10:	learn: 0.4413718	total: 631ms	remaining: 56.7s
11:	learn: 0.4375157	total: 687ms	remaining: 56.5s
12:	learn: 0.4350157	total: 741ms	remaining: 56.3s
13:	learn: 0.4325392	total: 796ms	remaining: 56s
14:	learn: 0.4301793	total: 847ms	remaining: 55.6s
15:	learn: 0.4285591	total: 899ms	remaining: 55.3s
16:	learn: 0.4268543	total: 951ms	remaining: 55s
17:	learn: 0.4248588	total: 1.01s	remaining: 54.9s
18:	learn: 0.4234115	total: 1.06s	remaining: 54.9s
19:	learn: 0.42229

162:	learn: 0.3851319	total: 8.89s	remaining: 45.6s
163:	learn: 0.3850483	total: 8.94s	remaining: 45.6s
164:	learn: 0.3849828	total: 9s	remaining: 45.5s
165:	learn: 0.3849212	total: 9.05s	remaining: 45.5s
166:	learn: 0.3848438	total: 9.11s	remaining: 45.5s
167:	learn: 0.3847645	total: 9.17s	remaining: 45.4s
168:	learn: 0.3846732	total: 9.23s	remaining: 45.4s
169:	learn: 0.3845724	total: 9.29s	remaining: 45.3s
170:	learn: 0.3844748	total: 9.34s	remaining: 45.3s
171:	learn: 0.3844117	total: 9.4s	remaining: 45.3s
172:	learn: 0.3842872	total: 9.46s	remaining: 45.2s
173:	learn: 0.3842041	total: 9.52s	remaining: 45.2s
174:	learn: 0.3841336	total: 9.58s	remaining: 45.2s
175:	learn: 0.3840586	total: 9.64s	remaining: 45.1s
176:	learn: 0.3839968	total: 9.7s	remaining: 45.1s
177:	learn: 0.3839219	total: 9.76s	remaining: 45.1s
178:	learn: 0.3838322	total: 9.81s	remaining: 45s
179:	learn: 0.3837579	total: 9.87s	remaining: 45s
180:	learn: 0.3836875	total: 9.93s	remaining: 44.9s
181:	learn: 0.3835868

321:	learn: 0.3757981	total: 17.8s	remaining: 37.5s
322:	learn: 0.3757717	total: 17.9s	remaining: 37.4s
323:	learn: 0.3757284	total: 17.9s	remaining: 37.4s
324:	learn: 0.3756765	total: 18s	remaining: 37.3s
325:	learn: 0.3756359	total: 18s	remaining: 37.3s
326:	learn: 0.3756006	total: 18.1s	remaining: 37.2s
327:	learn: 0.3755695	total: 18.1s	remaining: 37.1s
328:	learn: 0.3755024	total: 18.2s	remaining: 37.1s
329:	learn: 0.3754456	total: 18.2s	remaining: 37s
330:	learn: 0.3754076	total: 18.3s	remaining: 36.9s
331:	learn: 0.3753801	total: 18.3s	remaining: 36.9s
332:	learn: 0.3753304	total: 18.4s	remaining: 36.8s
333:	learn: 0.3752802	total: 18.4s	remaining: 36.8s
334:	learn: 0.3752407	total: 18.5s	remaining: 36.7s
335:	learn: 0.3751944	total: 18.5s	remaining: 36.6s
336:	learn: 0.3751628	total: 18.6s	remaining: 36.6s
337:	learn: 0.3751203	total: 18.7s	remaining: 36.5s
338:	learn: 0.3750731	total: 18.7s	remaining: 36.5s
339:	learn: 0.3750192	total: 18.8s	remaining: 36.4s
340:	learn: 0.3749

481:	learn: 0.3697614	total: 26.6s	remaining: 28.6s
482:	learn: 0.3697188	total: 26.7s	remaining: 28.6s
483:	learn: 0.3696872	total: 26.7s	remaining: 28.5s
484:	learn: 0.3696506	total: 26.8s	remaining: 28.4s
485:	learn: 0.3696204	total: 26.8s	remaining: 28.4s
486:	learn: 0.3695938	total: 26.9s	remaining: 28.3s
487:	learn: 0.3695704	total: 26.9s	remaining: 28.3s
488:	learn: 0.3695396	total: 27s	remaining: 28.2s
489:	learn: 0.3695151	total: 27.1s	remaining: 28.2s
490:	learn: 0.3694796	total: 27.1s	remaining: 28.1s
491:	learn: 0.3694500	total: 27.2s	remaining: 28s
492:	learn: 0.3694087	total: 27.2s	remaining: 28s
493:	learn: 0.3693963	total: 27.3s	remaining: 27.9s
494:	learn: 0.3693737	total: 27.3s	remaining: 27.9s
495:	learn: 0.3693524	total: 27.4s	remaining: 27.8s
496:	learn: 0.3693143	total: 27.4s	remaining: 27.8s
497:	learn: 0.3692894	total: 27.5s	remaining: 27.7s
498:	learn: 0.3692661	total: 27.5s	remaining: 27.6s
499:	learn: 0.3692374	total: 27.6s	remaining: 27.6s
500:	learn: 0.3692

640:	learn: 0.3653441	total: 35.2s	remaining: 19.7s
641:	learn: 0.3653133	total: 35.2s	remaining: 19.7s
642:	learn: 0.3652960	total: 35.3s	remaining: 19.6s
643:	learn: 0.3652868	total: 35.4s	remaining: 19.5s
644:	learn: 0.3652605	total: 35.4s	remaining: 19.5s
645:	learn: 0.3652332	total: 35.5s	remaining: 19.4s
646:	learn: 0.3652036	total: 35.5s	remaining: 19.4s
647:	learn: 0.3651651	total: 35.6s	remaining: 19.3s
648:	learn: 0.3651494	total: 35.6s	remaining: 19.3s
649:	learn: 0.3651164	total: 35.7s	remaining: 19.2s
650:	learn: 0.3651057	total: 35.7s	remaining: 19.2s
651:	learn: 0.3650885	total: 35.8s	remaining: 19.1s
652:	learn: 0.3650679	total: 35.8s	remaining: 19s
653:	learn: 0.3650427	total: 35.9s	remaining: 19s
654:	learn: 0.3650190	total: 36s	remaining: 18.9s
655:	learn: 0.3650056	total: 36s	remaining: 18.9s
656:	learn: 0.3649835	total: 36.1s	remaining: 18.8s
657:	learn: 0.3649607	total: 36.1s	remaining: 18.8s
658:	learn: 0.3649393	total: 36.2s	remaining: 18.7s
659:	learn: 0.364912

800:	learn: 0.3616221	total: 43.8s	remaining: 10.9s
801:	learn: 0.3616059	total: 43.9s	remaining: 10.8s
802:	learn: 0.3615865	total: 43.9s	remaining: 10.8s
803:	learn: 0.3615591	total: 44s	remaining: 10.7s
804:	learn: 0.3615376	total: 44s	remaining: 10.7s
805:	learn: 0.3615258	total: 44.1s	remaining: 10.6s
806:	learn: 0.3615135	total: 44.2s	remaining: 10.6s
807:	learn: 0.3614966	total: 44.2s	remaining: 10.5s
808:	learn: 0.3614752	total: 44.3s	remaining: 10.4s
809:	learn: 0.3614564	total: 44.3s	remaining: 10.4s
810:	learn: 0.3614353	total: 44.4s	remaining: 10.3s
811:	learn: 0.3614081	total: 44.4s	remaining: 10.3s
812:	learn: 0.3613801	total: 44.5s	remaining: 10.2s
813:	learn: 0.3613637	total: 44.5s	remaining: 10.2s
814:	learn: 0.3613462	total: 44.6s	remaining: 10.1s
815:	learn: 0.3613155	total: 44.6s	remaining: 10.1s
816:	learn: 0.3612897	total: 44.7s	remaining: 10s
817:	learn: 0.3612703	total: 44.8s	remaining: 9.96s
818:	learn: 0.3612410	total: 44.8s	remaining: 9.9s
819:	learn: 0.36122

960:	learn: 0.3585369	total: 52.4s	remaining: 2.13s
961:	learn: 0.3585143	total: 52.4s	remaining: 2.07s
962:	learn: 0.3585010	total: 52.5s	remaining: 2.02s
963:	learn: 0.3584723	total: 52.5s	remaining: 1.96s
964:	learn: 0.3584541	total: 52.6s	remaining: 1.91s
965:	learn: 0.3584402	total: 52.7s	remaining: 1.85s
966:	learn: 0.3584300	total: 52.7s	remaining: 1.8s
967:	learn: 0.3584058	total: 52.8s	remaining: 1.74s
968:	learn: 0.3583790	total: 52.8s	remaining: 1.69s
969:	learn: 0.3583546	total: 52.9s	remaining: 1.64s
970:	learn: 0.3583346	total: 52.9s	remaining: 1.58s
971:	learn: 0.3583090	total: 53s	remaining: 1.52s
972:	learn: 0.3582931	total: 53s	remaining: 1.47s
973:	learn: 0.3582773	total: 53.1s	remaining: 1.42s
974:	learn: 0.3582508	total: 53.1s	remaining: 1.36s
975:	learn: 0.3582340	total: 53.2s	remaining: 1.31s
976:	learn: 0.3582165	total: 53.2s	remaining: 1.25s
977:	learn: 0.3581967	total: 53.3s	remaining: 1.2s
978:	learn: 0.3581726	total: 53.3s	remaining: 1.14s
979:	learn: 0.3581

In [88]:
#f1 score
print('f1 score : {:0.4f}'.format(f1_score(y_val, cb_rus_preds)))
#정확도
print('accuracy: {:0.4f}'.format(accuracy_score(y_val, cb_rus_preds)))
# 정밀도
print('precision: {:0.4f}'.format(precision_score(y_val, cb_rus_preds)))
#재현율
print('recall: {:0.4f}'.format(recall_score(y_val, cb_rus_preds)))
# ROC_AUC
print('roc_auc: {:0.4f}'.format(roc_auc_score(y_val, cb_rus_preds)))

f1 score : 0.3610
accuracy: 0.8391
precision: 0.2329
recall: 0.8018
roc_auc: 0.8216


In [137]:
import catboost
def catboost_classifier(depth, l2_leaf_reg, num_boost_round):
    params = {
        "loss_function": "Logloss",
        "eval_metric": "F1", 
        "depth" : int(depth),
        "l2_leaf_reg" : int(l2_leaf_reg),
        "learning_rate" : 0.01,
        "random_state" : 42,
        "logging_level" : "Silent",
        "thread_count": 24,
        "num_boost_round": int(num_boost_round)
    }
    train_data = catboost.Pool(data=X_rus, label=y_rus)
    cv_result = catboost.cv(train_data,
                       params,
                       early_stopping_rounds=100,
                       stratified=True,
                       nfold=3)
    return max(cv_result['f1-mean'])

In [138]:
catboostBO = BayesianOptimization(catboost_classifier, {
                                                'depth': (1, 4),
                                                'l2_leaf_reg': (2, 30),
                                                'num_boost_round': (100, 1000),
                                                })

catboostBO.maximize(n_iter=10, init_points=2)

|   iter    |  target   |   depth   | l2_lea... | num_bo... |
-------------------------------------------------------------


KeyError: 'f1-mean'

#  